# Dependências

In [2]:
#!pip install psycopg2

In [3]:
#!pip install gensim==3.6.0
#!pip install binary gensim==3.6.0

In [4]:
#!pip3 uninstall numpy -y

In [5]:
#!pip install numpy

# Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import psycopg2 as postgres
import psycopg2.extras
import math
from gensim.models import Word2Vec

C:\Users\tield\anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# Funções

In [2]:
def connect():
    conn = None
    try:
        conn = postgres.connect(
            host="localhost",
            database="austin_test",
            user="postgres",
            password="root",
            port = 8000) #8000 - LACINA PC
                         #5432 - LOCALHOST
    except postgres.Error as e:
        print(e)
    return conn

In [3]:
def closeConnection(conn):
    sucess = False
    try:
        conn.close()
        sucess = True
    except postgres.Error as e:
        print(e)
    
    return sucess

In [4]:
def executeQuery(conn, sql):
    record = None
    try:
        #print(sql)
        cur = conn.cursor(cursor_factory = psycopg2.extras.RealDictCursor)
        cur.execute(sql)
        record = cur.fetchall()
        cur.close()
    except postgres.Error as e:
        print(e)
        cur.execute("ROLLBACK")
        cur.close()
    return record

In [5]:
def executeInsert(conn, sql):
    sucess = False
            
    try:
        #print(sql)
        cur = conn.cursor()
        cur.execute(sql)
        conn.commit()
        cur.close()
        sucess = True
    except postgres.Error as e:
        print(e)
        cur.execute("ROLLBACK")
        cur.close()

    return sucess

In [6]:
#Recebe um id e retorna as categorias e checkin do lugar
def getPOIInformation(conn, business_id,):
    
    sql = """
        SELECT checkin_count, name FROM pois_information WHERE id  = \'"""+str(business_id)+ """\'
    ;"""

    result = executeQuery(conn, sql)

    return result

In [7]:
#Encontra todos os pontos do bin centrado em um POI trazendo suas informações (categorias e checkin)
def getBinPOIsInformation(conn, business_id, bin_number):

    result = None

    sql = """
        SELECT fk_poi_id_context, name, checkin_count, distance_m 
        FROM bins_pois_information 
        WHERE fk_poi_id_center = \'"""+str(business_id)+"""\' AND fk_bin_number = """+str(bin_number)+""";"""

    result = executeQuery(conn, sql)

    return result


In [8]:
#Função que obtém as informações de uma view materializada referente ao OSM
def getBinOSMInformation(conn, business_id, bin_number, materialized_view):
    result = None

    sql = """
        SELECT *
        FROM """+materialized_view+"""
        WHERE id = \'"""+str(business_id)+"""\' AND number = """+str(bin_number)+""";"""

    #print (sql)

    result = executeQuery(conn, sql)

    return result


In [9]:
#Função que obtém as estatísticas do BIN como contagem de POIs e soma dos Checkins
def getBinStats(conn, business_id, binRange):

    sql = """
    SELECT count(business_id) as quantity, sum(checkin_count) as total_checkin FROM poi WHERE business_id IN (
        SELECT fk_poi_business_id_01 FROM has_distance WHERE distance_m >= """+str(binRange[0])+""" AND distance_m < """+str(binRange[1])+"""
                                            AND fk_poi_business_id_02 = \'"""+str(business_id)+"""\'
    )
    """

    result = executeQuery(conn, sql)

    return result

In [10]:
def getCategoryInformation(data, category):
   
    occurences = 0
    checkin = 0

     #Formato de data
    #[business_id, checkin, category]
    for item in data:
        if(item[2] == category):
            occurences = occurences + 1
            checkin = checkin + item[1]
    
    return [checkin, occurences]


ITDL - [POI, POI] (Versões em Disco e Memória)

In [11]:
#Versão que salva dados na memória até o fim do processamento, só então salva no disco
def calculateBin(df, bin_number, w=0.5):
    
    print("executing bin:", bin_number, "\tweight:", w)

    #Criando canal de comunicação com a base de dados
    connection = connect()  

    if (connection != None):

        #Dicionário do ITDL
        scITDL = {} 
        i = 0  

        # largura do bin

        #centros
        for id_01, poi in df.iterrows():
            #print('calculating point:', id_01, 'id:', poi['business_id'])

            #Obtendo informações de categorias e checkin do poi central
            #[business_id, checkin, category]
            poi_information = getPOIInformation(connection, poi['business_id'])

            

            #[business_id, checkin, category, distance_m]
            bin_information = getBinPOIsInformation(connection, poi['business_id'], bin_number)
            
            #Executa o processo se há dados
            if (len(poi_information) > 0 and len(bin_information) > 0):

                columns = list(dict(bin_information[0]).keys())

                bin_information = pd.DataFrame(bin_information, columns = columns)

                #Calculando os dois parâmetros abaixo

                #sc - check-in total of all place types in bin n
                #sp - POI total of all place types in bin n

                #sp = len(bin_information['fk_poi_id_context'].unique())
                sp = len(bin_information['name'])  #Contando todas as categorias que aparecem no bin

                #sc = bin_information.drop_duplicates(subset = 'fk_poi_id_context')['checkin_count'].sum()
                sc = bin_information['checkin_count'].sum()  #Somando todos os checkins/por categoria no bin


                #As adições são feitas baseadas nos rótulos

                #Para evitar divisão por zero
                if(sc != 0 and sp != 0):
                    for center_poi in poi_information: # Para cada tki
                        for id_02, row in bin_information.iterrows(): #Para cara tkj


                            #cc = all checkin o tkj
                            #cp = all occurences of tkj

                            cc = bin_information[bin_information['name'] == row['name']]['checkin_count'].sum()
                            cp = bin_information[bin_information['name'] == row['name']]['name'].count()

                            #print(sc, sp, cc, cp)

                            a = (1 - (cc/sc)) #Pode gerar 0
                            u = (cp/sp)       #Pode gerar 0

                            if((a > 0) and (u > 0)):

                                A = -np.log2(a)
                                U = -np.log2(u)

                                aug = int(math.ceil((w*A) + ((1 - w)*U)))

                                #Aumentando-o pelo fator b
                                for b in range(aug):

                                    scITDL[i] = {   'poi_id_center': poi['business_id'],
                                                    'center_poi': center_poi['name'],
                                                    'poi_id_context': row['fk_poi_id_context'],
                                                    'context_poi': row['name'], 
                                                    'distance-m': row['distance_m']}
                                    i = i + 1

                #break


        scITDL = pd.DataFrame.from_dict(scITDL, 'index')
        print(len(scITDL))
        name = './Austin/w05/ITDL Partial Bins/austin-sl-tuple-n-itdl-' + str(bin_number) + 'bin-wgt'+str(w)+'-p.parquet'
        scITDL.to_parquet(name, compression='brotli',  index = False)
        closeConnection(connection)
        
    return None

In [12]:
#Versão que abre arquivo no disco e salva durante o processamento
import csv
def calculateBin_Disco(df, bin_number, w=0.5):
    
    w = round(w, 1)
    
    print("executing bin:", bin_number, "\tweight:", w)

    #Arquivo para salvar diretamente no disco
    file_name = './Austin/geographic/ITDL Partial Bins/austin-sl-tuple-n-itdlcu-' + str(bin_number) + 'bin-wgt'+str(w)+'-p.csv'
    csv_file = open(file_name, "w", newline='')
    writer = csv.writer(csv_file, delimiter=',')
    writer.writerow(["poi_id_center","center_poi","poi_id_context","context_poi","distance-m"])

    #Criando canal de comunicação com a base de dados
    connection = connect()  

    if (connection != None):

        #centros
        for id_01, poi in df.iterrows():
            #print('calculating point:', id_01, 'id:', poi['business_id'])

            #Obtendo informações de categorias e checkin do poi central
            #[business_id, checkin, category]
            poi_information = getPOIInformation(connection, poi['business_id'])


            #[business_id, checkin, category, distance_m]
            bin_information = getBinPOIsInformation(connection, poi['business_id'], bin_number)
            
            #Executa o processo se há dados
            if (len(poi_information) > 0 and len(bin_information) > 0):

                columns = list(dict(bin_information[0]).keys())

                bin_information = pd.DataFrame(bin_information, columns = columns)

                #Calculando os dois parâmetros abaixo
                #sc - check-in total of all place types in bin n
                #sp - POI total of all place types in bin n

                #sp = len(bin_information['fk_poi_id_context'].unique())
                sp = len(bin_information['name'])  #Contando todas as categorias que aparecem no bin

                #sc = bin_information.drop_duplicates(subset = 'fk_poi_id_context')['checkin_count'].sum()
                sc = bin_information['checkin_count'].sum()  #Somando todos os checkins/por categoria no bin


                #As adições são feitas baseadas nos rótulos

                #Para evitar divisão por zero
                if(sc != 0 and sp != 0):
                    for center_poi in poi_information: # Para cada tki
                        for id_02, row in bin_information.iterrows(): #Para cara tkj


                            #cc = all checkin o tkj
                            #cp = all occurences of tkj

                            cc = bin_information[bin_information['name'] == row['name']]['checkin_count'].sum()
                            cp = bin_information[bin_information['name'] == row['name']]['name'].count()

                            #print(sc, sp, cc, cp)

                            a = (1 - (cc/sc)) #Pode gerar 0
                            u = (cp/sp)       #Pode gerar 0

                            if((a > 0) and (u > 0)):

                                A = -np.log2(a)
                                U = -np.log2(u)

                                aug = int(math.ceil((w*A) + ((1 - w)*U)))

                                #Aumentando-o pelo fator b
                                for b in range(aug):

                                    line = [str(poi['business_id']), str(center_poi['name']), str(row['fk_poi_id_context']), str(row['name']), str(row['distance_m'])]
                                    writer.writerow(line)

                            
            #break
        
        csv_file.close()
        closeConnection(connection)
        
    return None

In [13]:
#Versão que abre arquivo no disco e salva durante o processamento também considera a unicidade inversa com relação ao ITDL
import csv
def calculateBin_Disco_u(df, bin_number, w=0.5):
    
    print("executing bin:", bin_number, "\tweight:", w)

    #Arquivo para salvar diretamente no disco
    file_name = './Austin/geographic/ITDL Partial Bins/austin-sl-tuple-n-itdlu-' + str(bin_number) + 'bin-wgt'+str(w)+'-p.csv'
    csv_file = open(file_name, "w", newline='')
    writer = csv.writer(csv_file, delimiter=',')
    writer.writerow(["poi_id_center","center_poi","poi_id_context","context_poi","distance-m"])

    #Criando canal de comunicação com a base de dados
    connection = connect()  

    if (connection != None):

        #centros
        for id_01, poi in df.iterrows():
            #print('calculating point:', id_01, 'id:', poi['business_id'])

            #Obtendo informações de categorias e checkin do poi central
            #[business_id, checkin, category]
            poi_information = getPOIInformation(connection, poi['business_id'])


            #[business_id, checkin, category, distance_m]
            bin_information = getBinPOIsInformation(connection, poi['business_id'], bin_number)
            
            #Executa o processo se há dados
            if (len(poi_information) > 0 and len(bin_information) > 0):

                columns = list(dict(bin_information[0]).keys())

                bin_information = pd.DataFrame(bin_information, columns = columns)

                #Calculando os dois parâmetros abaixo
                #sc - check-in total of all place types in bin n
                #sp - POI total of all place types in bin n

                #sp = len(bin_information['fk_poi_id_context'].unique())
                sp = len(bin_information['name'])  #Contando todas as categorias que aparecem no bin

                #sc = bin_information.drop_duplicates(subset = 'fk_poi_id_context')['checkin_count'].sum()
                sc = bin_information['checkin_count'].sum()  #Somando todos os checkins/por categoria no bin


                #As adições são feitas baseadas nos rótulos

                #Para evitar divisão por zero
                if(sc != 0 and sp != 0):
                    for center_poi in poi_information: # Para cada tki
                        for id_02, row in bin_information.iterrows(): #Para cara tkj


                            #cc = all checkin o tkj
                            #cp = all occurences of tkj

                            cc = bin_information[bin_information['name'] == row['name']]['checkin_count'].sum()
                            cp = bin_information[bin_information['name'] == row['name']]['name'].count()

                            #print(sc, sp, cc, cp)

                            a = (1 - (cc/sc)) #Pode gerar 0
                            u = (1 - (cp/sp)) #Pode gerar 0

                            if((a > 0) and (u > 0)):

                                A = -np.log2(a)
                                U = -np.log2(u)

                                aug = int(math.ceil((w*A) + ((1 - w)*U)))

                                #Aumentando-o pelo fator b
                                for b in range(aug):

                                    line = [str(poi['business_id']), str(center_poi['name']), str(row['fk_poi_id_context']), str(row['name']), str(row['distance_m'])]
                                    writer.writerow(line)

                            
            #break
        
        csv_file.close()
        closeConnection(connection)
        
    return None

Seção de ITDL Geográfico - [POI, GEO] (Todos abaixo)

In [15]:
import csv
def calculateBinOSMPolygon_Disco(df, bin_number, w=0.5):
    
    w = round(w, 1)
    
    print("executing bin:", bin_number, "/tweight:", w)
    h = 100         #Predefinido no ITDL

    #Arquivo para salvar diretamente no disco
    #name = 'C:/Users/tield/Documents/Place2vec Experiments/Austin/w05/ITDL Osm Partial Bins/austin-sl-tuple-n-itdl-' + str(n) + 'bin-wgt'+str(w)+'-bins_polygons_information-p.csv'
    file_name = './Austin/geographic/ITDLOSM Partial Bins/austin-sl-tuple-n-itdlosm-' + str(bin_number) + 'bin-wgt'+str(w)+'-bins_polygons_information-p.csv'
    csv_file = open(file_name, "w", newline='')
    writer = csv.writer(csv_file, delimiter=',')
    writer.writerow(["poi_id_center","center_poi","osm_id_context","context_osm","distance-m"])

    #Criando canal de comunicação com a base de dados
    connection = connect()  

    if (connection != None):

        for id_01, poi in df.iterrows():
            #print('calculating point:', id_01, 'id:', poi['business_id'])

            #[business_id, checkin, category]
            poi_information = getPOIInformation(connection, poi['business_id'])

            #bin_osm_information = []
            #bin_osm_building_information = []

            #[business_id, checkin, category, distance_m]
            bin_osm_information = getBinOSMInformation(connection, poi['business_id'], bin_number, 'bins_polygons_information')
            bin_osm_building_information = getBinOSMInformation(connection, poi['business_id'], bin_number, 'bins_polygons_building_information')


            #Calculando os dois parâmetros abaixo
            #sc - área total de todos os tipos de polygons no bin
            #sp - total de tipos diferentes de polygons no bin
            sp = 0
            sc = 0

            #Se o bin está preenchido com alguma informação
            if (len(bin_osm_information) > 0):
                tags = list(dict(bin_osm_information[0]).keys())
                bin_osm_information = pd.DataFrame(bin_osm_information, columns = tags)
                
                #sp = sp + len(bin_osm_information['osm_id'].unique())
                #Verificando quantas tipos de polygons diferentes existem
                sp = sp + bin_osm_information.iloc[:,2:len(tags)-3][~bin_osm_information.iloc[:,2:len(tags)-3].isin(['None'])].count().sum()
                #sc = sc + bin_osm_information['way_area_m'].sum()
                #Verificando quantas tipos de polygons diferentes e somando suas áreas (para cada tipo de polygon)
                sc = sc + (bin_osm_information.iloc[:,2:len(tags)-3][~bin_osm_information.iloc[:,2:len(tags)-3].isin(['None'])].count(axis=1)*bin_osm_information['way_area_m']).sum()

                #Excluindo ids e bin_number
                tags = tags[2:len(tags)-3]

            #Se o bin está preenchido com alguma informação
            if (len(bin_osm_building_information) > 0):
                tags_buildings = list(dict(bin_osm_building_information[0]).keys())
                bin_osm_building_information = pd.DataFrame(bin_osm_building_information, columns = tags_buildings)
                
                sp = sp + bin_osm_building_information.iloc[0]['building_count']
                sc = sc + bin_osm_building_information.iloc[0]['area_total']

                #Excluindo ids e bin_number
                tags_buildings = tags_buildings[1:2]

            #As adições são feitas baseadas nos rótulos

            #Para evitar divisão por zero
            if(sc != 0 and sp != 0):
                for center_poi in poi_information: # Para cada tki

                    #Calculando a co-ocorrência com polígonos que não são unicamente prédios
                    if(len(bin_osm_information) > 0):
                        for id_02, row in bin_osm_information.iterrows(): #Para cara polygons
                            for tag in tags:
                                #Percorrer cada tag
                                
                                value = row[tag]
                                if (value != None): #Para tags vazias

                                    #cc = all area o tag
                                    #cp = all occurences of tag                               
                                    cc = bin_osm_information[bin_osm_information[tag] == value]['way_area_m'].sum()
                                    cp = bin_osm_information[bin_osm_information[tag] == value][tag].count()

                                    a = (1 - (cc/sc)) #Pode gerar 0
                                    u = (cp/sp)       #Pode gerar 0
                                    #u = (1 - (cp/sp))       #Pode gerar 0

                                    if((a > 0) and (u > 0)):

                                        A = -np.log2(a)
                                        U = -np.log2(u)

                                        aug = int(math.ceil((w*A) + ((1 - w)*U)))
                                        
                                        #print(tag, value)
                                        name = tag+"_"+value

                                        #Aumentando-o pelo fator b
                                        for b in range(aug):

                                            line = [str(poi['business_id']), str(center_poi['name']), str(row['osm_id']), str(name), str(row['distance_m'])]
                                            writer.writerow(line)

                    #Calculando a co-ocorrência com polígonos que são unicamente prédios                    
                    if (len(bin_osm_building_information) > 0):

                        for id_02, row in bin_osm_building_information.iterrows(): #Para cara buildings

                                
                                #cc = all area o tag
                                #cp = all occurences of tag

                                cc = row['area_total']
                                cp = row['building_count']

                                
                                a = (1 - (cc/sc)) #Pode gerar 0
                                u = (cp/sp)       #Pode gerar 0
                                #u = (1 - (cp/sp))       #Pode gerar 0

                                if((a > 0) and (u > 0)):

                                    A = -np.log2(a)
                                    U = -np.log2(u)

                                    aug = int(math.ceil((w*A) + ((1 - w)*U)))

                                    name = 'building_yes'
                                    distance = bin_number*h + (h/2)
                                    building_id = '-1'

                                    #Aumentando-o pelo fator b
                                    for b in range(aug):

                                        line = [str(poi['business_id']), str(center_poi['name']), building_id, str(name), str(distance)]
                                        writer.writerow(line)

                            
            #break
        
        
        csv_file.close()
        closeConnection(connection)
        
    return None

In [14]:
import csv
def calculateBinOSMRoadsLines_Disco(df, bin_number, roads=True, w=0.5):
    
    if(roads):
        materialized_view = 'bins_roads_information'
    else:
        materialized_view = 'bins_lines_information'

    print("executing bin:", bin_number, "\tweight:", w)


    #Arquivo para salvar diretamente no disco
    #name = 'C:/Users/tield/Documents/Place2vec Experiments/Austin/w05/ITDL Osm Partial Bins/austin-sl-tuple-n-itdl-' + str(n) + 'bin-wgt'+str(w)+"-"+materialized_view+'-p.csv'
    file_name = './Austin/geographic/ITDLOSM Partial Bins/austin-sl-tuple-n-itdlosm-' + str(bin_number) + 'bin-wgt'+str(w)+"-"+materialized_view+'-p.csv'
    csv_file = open(file_name, "w", newline='')
    writer = csv.writer(csv_file, delimiter=',')
    writer.writerow(["poi_id_center","center_poi","osm_id_context","context_osm","distance-m"])

    #Criando canal de comunicação com a base de dados
    connection = connect()  

    if (connection != None):

        
        #centros
        for id_01, poi in df.iterrows():
            #print('calculating point:', id_01, 'id:', poi['business_id'])

            #[business_id, checkin, category]
            poi_information = getPOIInformation(connection, poi['business_id'])


            #[business_id, checkin, category, distance_m]
            bin_osm_information = getBinOSMInformation(connection, poi['business_id'], bin_number, materialized_view)

            #Se o bin está preenchido com alguma informação
            if (len(bin_osm_information) > 0):
                tags = list(dict(bin_osm_information[0]).keys())

                bin_osm_information = pd.DataFrame(bin_osm_information, columns = tags)

                

                #Calculando os dois parâmetros abaixo
                #sc - comprimento total
                #sp - total de roads/lines no bin
                #sp = len(bin_osm_information['osm_id'].unique())
                #sc = bin_osm_information['length'].sum()

                #sp - total de roads/lines de cada tipo no bin
                sp = bin_osm_information.iloc[:,2:len(tags)-3][~bin_osm_information.iloc[:,2:len(tags)-3].isin(['None'])].count().sum()
                #sc - comprimento total de cada tipo de roads/lines no bin
                sc = (bin_osm_information.iloc[:,2:len(tags)-3][~bin_osm_information.iloc[:,2:len(tags)-3].isin(['None'])].count(axis=1)*bin_osm_information['length']).sum()

                #Excluindo ids e bin_number
                tags = tags[2:len(tags)-3]
            
                #As adições são feitas baseadas nos rótulos

                #Para evitar divisão por zero
                if(sc != 0 and sp != 0):
                    for center_poi in poi_information: # Para cada tki
                        for id_02, row in bin_osm_information.iterrows(): #Para cara tkj
                            for tag in tags:
                                #Percorrer cada tag
                                
                                value = row[tag]
                                if (value != None): #Para tags vazias
                                
                                    #cc = all length o tag
                                    #cp = all occurences of tag
                                    cc = bin_osm_information[bin_osm_information[tag] == value]['length'].sum()
                                    cp = bin_osm_information[bin_osm_information[tag] == value][tag].count()

                                    a = (1 - (cc/sc)) #Pode gerar 0
                                    #u = (cp/sp)       #Pode gerar 0
                                    u = (1 - (cp/sp))       #Pode gerar 0

                                    if((a > 0) and (u > 0)):

                                        A = -np.log2(a)
                                        U = -np.log2(u)

                                        aug = int(math.ceil((w*A) + ((1 - w)*U)))

                                        name = tag+"_"+value

                                        #Aumentando-o pelo fator b
                                        for b in range(aug):

                                            line = [str(poi['business_id']), str(center_poi['name']), str(row['osm_id']), str(name), str(row['distance_m'])]
                                            writer.writerow(line)

                                
            #break
        
        csv_file.close()
        closeConnection(connection)
        
    return None

In [15]:
import csv
def calculateBinOSMPoints_Disco(df, bin_number):
    
    print("executing bin:", bin_number)

    #Arquivo para salvar diretamente no disco
    #name = 'C:/Users/tield/Documents/Place2vec Experiments/Austin/w05/ITDL Osm Partial Bins/austin-sl-tuple-n-itdl-' + str(n) + '-bins_points_information-p.csv'
    file_name = './Austin/geographic/ITDLOSM Partial Bins/austin-sl-tuple-n-itdlosm-' + str(bin_number) + 'bin-bins_points_information-p.csv'
    csv_file = open(file_name, "w", newline='')
    writer = csv.writer(csv_file, delimiter=',')
    writer.writerow(["poi_id_center","center_poi","osm_id_context","context_osm","distance-m"])

    #Criando canal de comunicação com a base de dados
    connection = connect()  

    if (connection != None):

        for id_01, poi in df.iterrows():
            #print('calculating point:', id_01, 'id:', poi['business_id'])

            #[business_id, checkin, category]
            poi_information = getPOIInformation(connection, poi['business_id'])


            #[business_id, checkin, category, distance_m]
            bin_osm_information = getBinOSMInformation(connection, poi['business_id'], bin_number, 'bins_points_information')

            #Se o bin está preenchido com alguma informação
            if (len(bin_osm_information) > 0):

                tags = list(dict(bin_osm_information[0]).keys())
                bin_osm_information = pd.DataFrame(bin_osm_information, columns = tags)

               
                #sp - total de points no bin
                #sp = len(bin_osm_information['osm_id'].unique())

                #sp - total de tipos diferentes de points no bin
                sp = bin_osm_information.iloc[:,2:len(tags)-2][~bin_osm_information.iloc[:,2:len(tags)-2].isin(['None'])].count().sum()
                 #Excluindo ids e bin_number
                tags = tags[2:len(tags)-2]
                #print(sp)
                
                #As adições são feitas baseadas nos rótulos

                #Para evitar divisão por zero
                if(sp != 0):
                    for center_poi in poi_information: # Para cada tki
                        for id_02, row in bin_osm_information.iterrows(): #Para cara tkj
                            for tag in tags:
                                #Percorrer cada tag
                                
                                value = row[tag]
                                if (value != None): #Para tags vazias
                                
                                    cp = bin_osm_information[bin_osm_information[tag] == value][tag].count()
                                    u = (cp/sp)       #Pode gerar 0
                                    #u = (1 - (cp/sp))       #Pode gerar 0

                                    if(u > 0):

                                        U = -np.log2(u)
                                        aug = int(math.ceil(U))

                                        name = tag+"_"+value

                                        #Aumentando-o pelo fator b
                                        for b in range(aug):

                                            line = [str(poi['business_id']), str(center_poi['name']), str(row['osm_id']), str(name), str(row['distance_m'])]
                                            writer.writerow(line)
        
            #break
        
        csv_file.close()
        closeConnection(connection)
        
    return None

ITDL - [POI, POI] - Utilizando Embeddings Geográficos

In [16]:
#Essa versão considera os checkins, unicidade e geografia do lugar para utilizar no fator de aumento da relação poi-poi
import csv
def calculateCGUBin_Disco(df, bin_number, main_parameter = 'checkin', w=0.5):

    #w = round(w, 1)
    
    #Ajuste dos valores dos parâmetros para os três elementos
    w_a = 0
    w_u = 0
    w_g = 0

    
    if (main_parameter == 'checkin'):
        w_a = w
        w_u = w_g = (1 - w_a)/2
    elif (main_parameter == 'geographic'):
        w_g = w
        w_u = w_a = (1 - w_g)/2
    else:
        w_u = w
        w_g = w_a = (1 - w_u)/2

    print("executing bin:", bin_number, "\tcheckin weight:", w_a, '\tproability weight:', w_u, '\tgeographic weight:', w_g)

    #Carregando vector embeddings geográficos
    model_name = './Austin/geographic/ITDLOSM Partial Bins/austin-sl-tuple-n-itdlosm-'+str(0)+'bin-wgt0.5-points-polygons.model'
    model = Word2Vec.load(model_name)

    #Arquivo para salvar diretamente no disco
    file_name = './Austin/geographic/ITDLG Partial Bins 2/austin-sl-tuple-n-itdlcgu-' + str(bin_number) + 'bin-'+main_parameter+'-wgt'+str(w)+'-p.csv'
    csv_file = open(file_name, "w", newline='')
    writer = csv.writer(csv_file, delimiter=',')
    writer.writerow(["poi_id_center","center_poi","poi_id_context","context_poi","distance-m"])

    #Criando canal de comunicação com a base de dados
    connection = connect()  

    if (connection != None):

        #centros
        for id_01, poi in df.iterrows():
            #print('calculating point:', id_01, 'id:', poi['business_id'])

            #Obtendo informações de categorias e checkin do poi central
            #[business_id, checkin, category]
            poi_information = getPOIInformation(connection, poi['business_id'])


            #[business_id, checkin, category, distance_m]
            bin_information = getBinPOIsInformation(connection, poi['business_id'], bin_number)
            
            #Executa o processo se há dados
            if (len(poi_information) > 0 and len(bin_information) > 0):

                columns = list(dict(bin_information[0]).keys())

                bin_information = pd.DataFrame(bin_information, columns = columns)

                #Calculando os dois parâmetros abaixo
                #sc - check-in total of all place types in bin n
                #sp - POI total of all place types in bin n

                #sp = len(bin_information['fk_poi_id_context'].unique())
                sp = len(bin_information['name'])  #Contando todas as categorias que aparecem no bin

                #sc = bin_information.drop_duplicates(subset = 'fk_poi_id_context')['checkin_count'].sum()
                sc = bin_information['checkin_count'].sum()  #Somando todos os checkins/por categoria no bin


                #As adições são feitas baseadas nos rótulos

                #Para evitar divisão por zero
                if(sc != 0 and sp != 0):
                    for center_poi in poi_information: # Para cada tki
                        for id_02, row in bin_information.iterrows(): #Para cara tkj


                            #cc = all checkin o tkj
                            #cp = all occurences of tkj

                            cc = bin_information[bin_information['name'] == row['name']]['checkin_count'].sum()
                            cp = bin_information[bin_information['name'] == row['name']]['name'].count()
                            
                            try:
                                sm = model.wv.similarity(center_poi['name'], row['name'])
                            except:
                                #Provável caso de falta de vocabulário
                                sm = 0
                            #print(sc, sp, cc, cp)

                            a = (1 - (cc/sc)) 
                            u = (cp/sp)       
                            g = (1 - sm)        #Pode gerar 0


                            if((a > 0) and (u > 0)):

                                A = -np.log2(a)
                                U = -np.log2(u)
                                
                                #log2(0) = inf
                                if (g > 0):
                                    G = -np.log2(g)
                                else:
                                    G = 0

                                #print('cc:', cc, ' sc:', sc, ' cc/sc:', a, ' a:', A)
                                #print('cp:', cp, ' sp:', sp, ' cp/sp:', u, ' u:', U)
                                #print('sm:', sm, ' g:', G)
                                #print()

                                aug = int(math.ceil((w_a*A) + (w_u*U) + (w_g*G)))
                                #print(aug)

                                #Aumentando-o pelo fator b
                                for b in range(aug):

                                    line = [str(poi['business_id']), str(center_poi['name']), str(row['fk_poi_id_context']), str(row['name']), str(row['distance_m'])]
                                    writer.writerow(line)
      
            #break
        
        csv_file.close()
        closeConnection(connection)
        
    return None

In [17]:
#Essa versão considera apenas os vetores geográficos e a unicidade de cada lugar, descartando os checkins para aumentar a relação poi-poi
import csv
def calculateGUBin_Disco(df, bin_number, w=0.5):

    w = round(w, 1)
    print("executing bin:", bin_number, '\tproability weight:', (1-w), '\tgeographic weight:', w)

    #Carregando vector embeddings geográficos
    model_name = './Austin/geographic/ITDLOSM Partial Bins/austin-sl-tuple-n-itdlosm-'+str(0)+'bin-wgt0.5-points-polygons.model'
    model = Word2Vec.load(model_name)

    #Arquivo para salvar diretamente no disco
    file_name = './Austin/geographic/ITDLG Partial Bins 2/austin-sl-tuple-n-itdlgu-' + str(bin_number) + 'bin-wgt'+str(w)+'-p.csv'
    csv_file = open(file_name, "w", newline='')
    writer = csv.writer(csv_file, delimiter=',')
    writer.writerow(["poi_id_center","center_poi","poi_id_context","context_poi","distance-m"])

    #Criando canal de comunicação com a base de dados
    connection = connect()  

    if (connection != None):

        #centros
        for id_01, poi in df.iterrows():
            #print('calculating point:', id_01, 'id:', poi['business_id'])

            #Obtendo informações de categorias e checkin do poi central
            #[business_id, checkin, category]
            poi_information = getPOIInformation(connection, poi['business_id'])


            #[business_id, checkin, category, distance_m]
            bin_information = getBinPOIsInformation(connection, poi['business_id'], bin_number)
            
            #Executa o processo se há dados
            if (len(poi_information) > 0 and len(bin_information) > 0):

                columns = list(dict(bin_information[0]).keys())

                bin_information = pd.DataFrame(bin_information, columns = columns)

                #Calculando os dois parâmetros abaixo
                #sc - check-in total of all place types in bin n
                #sp - POI total of all place types in bin n

                #sp = len(bin_information['fk_poi_id_context'].unique())
                sp = len(bin_information['name'])  #Contando todas as categorias que aparecem no bin

                #sc = bin_information.drop_duplicates(subset = 'fk_poi_id_context')['checkin_count'].sum()
                #sc = bin_information['checkin_count'].sum()  #Somando todos os checkins/por categoria no bin


                #As adições são feitas baseadas nos rótulos

                #Para evitar divisão por zero
                if(sp != 0):
                    for center_poi in poi_information: # Para cada tki
                        for id_02, row in bin_information.iterrows(): #Para cara tkj


                            #cc = all checkin o tkj
                            #cp = all occurences of tkj

                            #cc = bin_information[bin_information['name'] == row['name']]['checkin_count'].sum()
                            cp = bin_information[bin_information['name'] == row['name']]['name'].count()
                            
                            try:
                                sm = model.wv.similarity(center_poi['name'], row['name'])
                            except:
                                #Provável caso de falta de vocabulário
                                sm = 0
                            #print(sc, sp, cc, cp)

                            #a = (1 - (cc/sc)) 
                            u = (cp/sp)       
                            g = (1 - sm)        #Pode gerar 0


                            if((u > 0)):
         
                                U = -np.log2(u)
                                
                                #log2(0) = inf
                                if(g > 0):
                                    G = -np.log2(g)
                                else:
                                    G = 0

                                #print('cc:', cc, ' sc:', sc, ' cc/sc:', a, ' a:', A)
                                #print('cp:', cp, ' sp:', sp, ' cp/sp:', u, ' u:', U)
                                #print('sm:', sm, ' g:', G)
                                #print()

                                aug = int(math.ceil((w*G) + ((1 - w)*U)))

                                #Aumentando-o pelo fator b
                                for b in range(aug):

                                    line = [str(poi['business_id']), str(center_poi['name']), str(row['fk_poi_id_context']), str(row['name']), str(row['distance_m'])]
                                    writer.writerow(line)
      
            #break
        
        csv_file.close()
        closeConnection(connection)
        
    return None

Context - POI
Funções para encontrar o tipo do POI central a partir de elementos de contexto

In [25]:
def getCenterPOIByContext(df, bin_number, wgt, top = 10, context_type = 'itdl'):


    #Criando canal de comunicação com a base de dados
    connection = connect()  

    if (connection != None):

        #Selecionando um conjunto para teste
        
        df = df.iloc[0:99, :]

        #Carregando vector embeddings geográficos

        if (context_type == 'osm'):
            model_name = 'C:/Users/tield/Documents/Doutorado UFCG/Place2vec Experiments/Austin/Geographic/ITDL-OSM Partial Bins/austin-sl-tuple-n-itdlosm-'+str(0)+'bin-wgt'+str(wgt)+'-points-polygons.model'
            model = Word2Vec.load(model_name)
        elif (context_type == 'geographic'):
            model_name = 'C:/Users/tield/Documents/Doutorado UFCG/Place2vec Experiments/Austin/Geographic/ITDL-CGU Partial Bins/austin-sl-tuple-n-itdlcgu-'+str(0)+'bin-geographic-wgt'+str(wgt)+'-p.model'
            model = Word2Vec.load(model_name)
        elif (context_type == 'itdl'):
            model_name = 'C:/Users/tield/Documents/Doutorado UFCG/Place2vec Experiments/Austin/Geographic/ITDL Partial Bins/austin-sl-tuple-n-itdlcu-'+str(0)+'bin-wgt'+str(wgt)+'-p.model'
            model = Word2Vec.load(model_name)
        else:
            model_name = 'C:/Users/tield/Documents/Doutorado UFCG/Place2vec Experiments/Austin/Geographic/ITDL-GU Partial Bins/austin-sl-tuple-n-itdlgu-'+str(0)+'bin-wgt'+str(wgt)+'-p.model'
            model = Word2Vec.load(model_name)

        #centros
        match_count = 0
        poi_count = 0
        for id, poi in df.iterrows():

            
            #Obtendo informações de um determinado POI
            #[business_id, checkin, category]
            poi_information = getPOIInformation(connection, poi['business_id'])
  
            #Obtendo POIs de contexto
            if (context_type != 'osm'):

                #Obtendo informações do bin de um POI
                #[business_id, checkin, category, distance_m]
                bin_information = getBinPOIsInformation(connection, poi['business_id'], bin_number)

                if (len(poi_information) > 0 and len(bin_information) > 0):

                    poi_count = poi_count + 1

                    columns = list(dict(poi_information[0]).keys())
                    poi_information = pd.DataFrame(poi_information, columns = columns)

                    columns = list(dict(bin_information[0]).keys())
                    bin_information = pd.DataFrame(bin_information, columns = columns)

                    #Definindo intervalo de folga para considerar uma região dentro do bin
                    center_region = bin_information[bin_information['distance_m'] <=  (((bin_number+1)*100)/2)]
                    outer_region = bin_information[bin_information['distance_m'] > (((bin_number+1)*100)/2)]

                    #poi types em determinada região
                    center_poi_types = set(poi_information['name'].unique())
                    context_poi_types = set(center_region['name'].unique())
                    center_poi_types.update(context_poi_types)

                    #Obtendo elementos do contexto da região
                    context_elements = set(outer_region['name'].unique())
                    
                    #Predizendo possíveis pois centrais a partir do contexto
                    predicted_pois = model.predict_output_word(context_elements, topn=top)

                    #Não houve elementos para predição
                    if (predicted_pois != None):

                        predicted_center_pois = []
                        
                        for center_poi in predicted_pois:
                            predicted_center_pois.append(center_poi[0])

                        predicted_center_pois = set(predicted_center_pois)

                        #Avaliando quantos matchs ouveram
                        match = center_poi_types.intersection(predicted_center_pois)

                        if(len(match) > 0):
                            match_count = match_count + 1

                #Obtendo elementos geográficos de contexto
            """else:

                bin_osm_points_information = getBinOSMInformation(connection, poi['business_id'], bin_number, 'bins_points_information')
                bin_osm_polygons_information = getBinOSMInformation(connection, poi['business_id'], bin_number, 'bins_polygons_information')
                bin_osm_buildings_information = getBinOSMInformation(connection, poi['business_id'], bin_number, 'bins_polygons_building_information')
                bin_osm_lines_information = getBinOSMInformation(connection, poi['business_id'], bin_number, 'bins_lines_information')
                bin_osm_roads_information = getBinOSMInformation(connection, poi['business_id'], bin_number, 'bins_roads_information')

            """

    print("match count:", match_count)
    print("poi count:",poi_count)
    print("accuracy:", match_count/poi_count)

getCenterPOIByContext(df, 0, 0.5, top = 15, context_type = 'itdl')
                     

match count: 13
poi count: 91
accuracy: 0.14285714285714285


In [26]:
getCenterPOIByContext(df, 0, 0.5, top = 15, context_type = 'geographic')

match count: 18
poi count: 91
accuracy: 0.1978021978021978


In [27]:
getCenterPOIByContext(df, 0, 0.5, top = 15, context_type = 'gu')

match count: 12
poi count: 91
accuracy: 0.13186813186813187


# Geração do ITDL

In [18]:
#Quantidade de tuplas de vizinhos nos dados do yelp (center, context) considerando d = 100m
#df = pd.read_csv('/kaggle/input/yelpcambridge/yelpcambridge-ml.csv')
pois_file_name = 'C:/Users/tield/Documents/Doutorado UFCG/Place2vec Experiments/Cities/austin-ml-updated.csv'
df = pd.read_csv(pois_file_name)
print(df.shape)
df = df.dropna()
print(df.shape)
df.head()

(22399, 7)
(22399, 7)


,business_id,city,state,latitude,longitude,categories,checkin_count
0,N3_Gs3DnX4k9SgpwJxdEfw,Austin,TX,30.346169,-97.711458,"Shopping, Jewelry Repair, Appraisal Services, ...",14
1,tXvdYGvlEceDljN8gt2_3Q,Austin,TX,30.172706,-97.799920,"Barbers, Beauty & Spas",1
2,nTIhpR7MhsALPwg_Hh14EA,Austin,TX,30.326377,-97.704543,"Hotels, Hotels & Travel, Event Planning & Serv...",475
3,8XyEpVdAO0o6iVkVxkWosQ,Austin,TX,30.246465,-97.778738,"Home Services, Real Estate, Property Management",0
4,NVfOn7TdnHbaGH97CVB_Qg,Austin,TX,30.244902,-97.857409,"Chiropractors, Health & Medical",33


Iteractive ITDL

In [51]:
for n in range(0, 1):
    #res = calculateBin_Disco(df, n, 0.5)
    #res = calculateCGUBin_Disco(df, n, main_parameter='checkin', w=0.333333)
    getCenterPOIByContext(df, n, 0.5, context_type = 'pois')
    #calculateBinOSMPoints_Disco(df, n)
    #break

10
43
0.23255813953488372


In [ ]:
print('Ok')

In [71]:
for n in range(0, 1):
    #calculateBinOSMPolygon_Disco(df, n, 0.5)
    #calculateBinOSMRoadsLines_Disco(df, n, roads=False, w=0.5)
    calculateBinOSMPoints_Disco(df, n)
    #calculateBinOSM_Disco(df, n, 'bins_points_information', 0.5)
    #calculateBinOSM_Disco(df, n, 'bins_lines_information', 0.5)
    #calculateBinOSM_Disco(df, n, 'bins_roads_information', 0.5)

executing bin: 0 	weight: 0.5


In [ ]:
n = 0
for w in range(0, 1, 0.1):
    calculateGeoBin_Disco(df, n, main_parameter='geographic', w=w)

In [88]:
test = pd.read_parquet('austin-sl-tuple-n-itdl-0bin-wgt0.7-p.parquet')
print(test.shape)
test.head()

(2291, 3)


,centerPoI,contextPoI,distance-m
0,Shopping,Screen Printing,13.1
1,Shopping,Screen Printing,13.1
2,Shopping,Screen Printing,13.1
3,Shopping,Screen Printing/T-Shirt Printing,13.1
4,Shopping,Screen Printing/T-Shirt Printing,13.1


ITDL em Paralelo

In [16]:
import multiprocessing as mp
print("Number of processors: ", mp.cpu_count())

# Step 1: Init multiprocessing.Pool()
pool = mp.Pool(int(mp.cpu_count()/2))

# Step 2: `pool.apply` the `howmany_within_range()`
bins = range(0, 4)
#bins = [0]
#[pool.apply(calculateBin, args=(df, n, 100, 0.3)) for n in bins]
pool.starmap(calculateBin, [(df, n, 100, 0.7) for n in bins])

# Step 3: Don't forget to close
pool.close()

Number of processors:  8


In [ ]:
import multiprocessing as mp
print("Number of processors: ", mp.cpu_count())

# Step 1: Init multiprocessing.Pool()
pool = mp.Pool(int(mp.cpu_count()-4))

# Step 2: `pool.apply` the `howmany_within_range()`
weights = np.arange(0.0, 1.1, 0.1)
n = 8

#pool.starmap(calculateBinOSMPolygon_Disco, [(df, 0,  w) for w in weights])
#pool.starmap(calculateBinOSMRoadsLines_Disco, [(df, 0, True,  w) for w in weights])
#pool.starmap(calculateBinOSMRoadsLines_Disco, [(df, 0, roads=True,  w) for w in weights])

#pool.starmap(calculateBin_Disco, [(df, n, w) for w in weights])
pool.starmap(calculateCGUBin_Disco, [(df, n, 'geographic', w) for w in weights])
#pool.starmap(calculateGUBin_Disco, [(df, n,  w) for w in weights])


# Step 3: Don't forget to close
pool.close()

Number of processors:  16
executing bin:executing bin:executing bin:executing bin:executing bin:   

In [24]:
weights = np.arange(0.0, 1.1, 0.1)
weights

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])